In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn import datasets
from sklearn import svm
from sklearn import metrics

### Make data

In [2]:
numcat = 2
categories = ['healthy', 'disease']

In [3]:
df_h = pd.read_csv('health_data.csv')
print(df_h.head())

train_per = 0.7 #train test split
#randomize indces, take the first 80% of the indeces and last 20 percent as test
indices = np.random.permutation(df_h.shape[0])
train_ind, test_ind = indices[:int(train_per*df_h.shape[0])], indices[int(train_per*df_h.shape[0]):]
# print(len(train_ind), len(test_ind))

#select the data corresponding to the train and test indices and save into 2 dataframes. Reset index afterwards
train_df, test_df = df_h.loc[train_ind, :], df_h.loc[test_ind, :]
train_df, test_df = train_df.reset_index(drop = True), test_df.reset_index(drop = True)
# train_df.drop('index')

# Data in numpy arrays (also separating train data by classes)
X_train = train_df.drop('category', axis = 1).to_numpy()
y_train = train_df.drop(['age', 'restbps', 'chol'], axis = 1).to_numpy().reshape((X_train.shape[0],))

# X_train_0, X_train_1 = train_df.loc[train_df['category'] == 0].drop('category', axis = 1).to_numpy(),train_df.loc[train_df['category'] == 1].drop('category', axis = 1).to_numpy()
X_test, y_test = test_df.drop('category', axis = 1).to_numpy(), test_df['category'].to_numpy().reshape((-1, ))
print(X_test.shape, y_test.shape, X_train.shape, y_train.shape)

   age  restbps  chol  category
0   26      109   243         0
1   27      106   156         0
2   28      107   225         0
3   27      105   277         0
4   30       96   221         0
(211, 3) (211,) (489, 3) (489,)


In [4]:
class SVM_solver():
    def __init__(self, ker_type = 'linear', itermax = 1000, C= 10, epsilon = 0.0001, gamma = 10, poly = 1):
        self.kernel_type = ker_type
        self.C = C
        self.epsilon = epsilon
        self.gamma = gamma
        self.kernels = {'linear': self.k_linear, 'gaussian': self.k_gaussian, 'polynomial': self.k_polynomial}
        self.itermax = itermax
        self.poly = poly
        self.gamma = gamma
    
    def solve_SVM(self, X, y):
#         X.shape = (N, D) y.shape = (N,)
        N = X.shape[0]
        self.X = X
        y = 2*y-1
#         print(y)
        self.y = y
#         print(self.y)
#         print(self.y)
        K = self.get_ker_mat(self.X, y, self.kernels[self.kernel_type])
        
        #initializing mu's and b
        mu = np.zeros(N)
        b = 0
        
        for count in range(self.itermax):
            #outer loop, currently just looping over the all the mu's but some heuristics
            #can be used for selecting good values of mu_i's and mu_j's
            
            #saving the old values of mu to check for convergence
            mu_old = np.copy(mu)

            for i in range(N):
#                 print(b)
                xi, yi = X[i,:], y[i]
                E_i = self.calc_E(y, yi, mu, b, K[i])
                if(self.KKTviolate(mu[i], E_i, yi) == True):
                    j = self.choose_random(0, N, i)
                    mu_i_old, mu_j_old = np.copy(mu[i]), np.copy(mu[j])

                    xj, yj = X[j,:], y[j]
                    E_j = self.calc_E(y, yj, mu, b, K[j])
                    L, H = self.getLH(yi, yj, mu[i], mu[j], self.C)
                    
                    #skip if L and H are same
                    if(L == H):
                        continue
                    
                    eta = 2*K[i, j]-K[i, i]-K[j, j]
#                     print(eta)
                    if(eta==0):
                        continue
#                     print(eta)
                    mu[j] = mu_j_old - yj*(E_i-E_j)/eta
#                     print('eta = {}'.format(eta))
                    mu[j] = self.clip(L, mu[j] , H)
                    
                    #if not much change in mu_j then donot update this i
                    if(np.absolute(mu[j] - mu_j_old)<1e-5):
                        continue
                    
                    mu[i] = mu_i_old + yi*yj*(mu_j_old - mu[j])
                    b = self.get_b(b, E_i, E_j, yi, yj, xi, xj, K[i, i], K[j, j], K[i, j], mu[j], mu_j_old, mu[i], mu_i_old)
            
            
            #convergence criteria
#             if(np.absolute(mu_old - mu).sum() <= self.epsilon):
#                 break
                
            if(count%100==0):
                print('iteration number = {}'.format(count))
        return mu, b
    
    def get_ker_mat(self, X, y, Kernel):
        mat = np.zeros((X.shape[0], X.shape[0]))
        for i in range(X.shape[0]):
            for j in range(X.shape[0]):
                mat[i, j] = Kernel(X[i], X[j])
        return mat
    
    def get_b(self, b, E_i, E_j, yi, yj, xi, xj, Ki, Kj, Kij, mu_j_new, mu_j_old, mu_i_new, mu_i_old):
        b1 = b - E_i - yi*(mu_i_new-mu_i_old)*Ki - yj*(mu_j_new-mu_j_old)*Kij
        b2 = b - E_j - yj*(mu_j_new-mu_j_old)*Kj - yi*(mu_i_new-mu_i_old)*Kij
        
        if(mu_i_new<self.C and mu_i_new>0):
            b = b1
        elif(mu_j_new<self.C and mu_j_new>0):
            b = b2
        else:
            b = (b1+b2)/2
        return b
        
    
    def clip(self, L, mu, H):
#         print('mu = {}, clipmu = {}'.format(mu, np.clip(mu, L, H)))
        return np.clip(mu, L, H)        
    
    def getLH(self, yi, yj, mui, muj, c):
        if(yi!=yj):
            return max(0, muj-mui), min(c, c+muj-mui)
        if(yi==yj):
            return max(0, muj+mui-c), min(c, muj+mui)
        
    
    def choose_random(self, l, h, i):
        random = i
        
        while(random == i):
            random = np.random.randint(l, h)
        
        return random
    
    def KKTviolate(self, mui, Ei, yi):
        if((Ei*yi<-1*1e-5 and mui<self.C) or (Ei*yi > 1e-5 and mui>0)):
            return True
        else:
            return False
    
    def calc_E(self, y, yi, mu, b, Ki):
        assert(y.shape == mu.shape == Ki.shape)
        f = np.sum(mu*y*Ki) + b
        
        return f-yi  
    
    def k_linear(self, x1, x2):
#         print(x1.shape)
        return np.sum(x1*x2)
        
    def k_gaussian(self, x1, x2):
        return np.exp(-self.gamma*np.sum((x1 - x2)**2))
        
    def k_polynomial(self, x1, x2):
        return (1+np.sum(x1*x2))**self.p
    
    def get_kermat_test(self, X_test, X_train):
        mat = np.zeros((X_test.shape[0], X_train.shape[0]))
        Kernel = self.kernels[self.kernel_type]
        for i in range(X_test.shape[0]):
            for j in range(X_train.shape[0]):
                mat[i, j] = Kernel(X_test[i], X_train[j])
        return mat
    
    def predict(self, X_test, mu, b):
        yhat = np.zeros((X_test.shape[0]))
        k_test_mat = self.get_kermat_test(X_test, self.X)
        for i in range(X_test.shape[0]):
            fi = np.sum(mu*self.y*k_test_mat[i]) + b
            if(fi<0):
                yhat[i] = 0
            else:
                yhat[i] = 1
        return yhat

#     def getK():
        
        

In [5]:
#Note for RBF KERNEL gamma = 1/2sigma^2
svm_model = SVM_solver(ker_type = 'linear', itermax = 1000, C = 1, epsilon = 0.001, gamma = 1, poly = 1)# gamma will only be used for the gaussian kernel
mu, b = svm_model.solve_SVM(X_train, y_train)
y_hat_test = svm_model.predict(X_test, mu, b)
# get_stats(y_hat_test, y_test)

iteration number = 0
iteration number = 100
iteration number = 200
iteration number = 300
iteration number = 400
iteration number = 500
iteration number = 600
iteration number = 700
iteration number = 800
iteration number = 900


In [15]:
# print(y_hat_test)
print((y_hat_test==y_test).sum()/(y_test.shape[0]))

0.8625592417061612


In [14]:
# y_hat_train = svm_model.predict(X_train, mu, b)


In [13]:
# print((y_hat_train==y_train).sum()/(y_train.shape[0]))

In [12]:
# print(y_hat_train)

In [11]:
# print(mu, b)

### Comparison of LibSVM and SMO algorithm implemented above

In [18]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [19]:
y_pred_test = clf.predict(X_test)

In [20]:
print((y_pred_test==y_test).sum()/(y_test.shape[0]))

0.8720379146919431
